In [1]:
import torch
import torch.nn as nn
import os
from ode_nn import AutoODE_COVID, weight_fun
from ode_nn import Dataset, train_epoch, eval_epoch, get_lr
import numpy as np
import pandas as pd
import torch.nn.functional as F
from torch.utils import data
import matplotlib.pyplot as plt
import random
import warnings
from ode_nn import Dataset_graph, train_epoch_graph, eval_epoch_graph, get_lr
warnings.filterwarnings("ignore")
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

Using backend: pytorch


In [2]:
from os.path import join
from pathlib import Path
os.getcwd()

'/home/vislab/Yejin/AutoODE-DSL'

## Data Preprocessing

In [3]:
# Read and Preprocess the csv files from John Hopkins Dataset
# https://github.com/CSSEGISandData/COVID-19/tree/master/csse_covid_19_data/csse_covid_19_daily_reports_us
# direc = ".../ODEs/Data/COVID/" # Directory that contains daily report csv files.
direc = join(os.getcwd(), 'ODEs', 'Data', 'COVID')
list_csv = sorted(os.listdir(direc))
us = []
for file in list_csv:
    sample = pd.read_csv(join(direc, file)).set_index("Province_State")[["Confirmed", "Recovered", "Deaths"]].sort_values(by = "Confirmed", ascending = False)
    us.append(sample.drop(['Diamond Princess', 'Grand Princess']))
us = pd.concat(us, axis=1, join='inner')
us_data = us.values.reshape(56,-1,3)
us_data[us_data!=us_data] = 0

#####################################################################################
# Normalize by total population of each state
population_path = join(os.getcwd(), 'ode_nn', 'population_states.csv')
population = pd.read_csv(population_path, index_col=0)

scaler = population.loc[us.index].values.reshape(56, 1, 1)*1e6
us_data = us_data/scaler
us_data = torch.from_numpy(us_data).float().to(device)

# Mobility Data: beta = 1 - stay_at_home_percentages
mobility_path = join(os.getcwd(), 'ode_nn', 'mobility')
beta = torch.load(join(mobility_path, 'us_beta.pt')).float().to(device)

# U.S states 1-0 Adjacency Matrix
graph = torch.load(join(mobility_path,'us_graph.pt')).float().to(device)

## Train AutoODE-COVID

In [4]:
##################################################################
test_idx = 131

# Learning Rate
lr = 0.01

# number of historic data points for fitting
input_steps = 10 

# forecasting horizon
output_steps = 7

# number of epochs for training
num_epochs = 20000

# select data for training
data = us_data[:, test_idx-input_steps:test_idx+7].to(device)
y_exact = data[:,:input_steps].to(device)

##################################################################

model = AutoODE_COVID(initial_I = data[:,0,0], initial_R = data[:,0,1], initial_D = data[:,0,2],
                      num_regions = 56, solver = "RK4", n_breaks = 1, graph = graph).to(device)

optimizer = torch.optim.Adam(model.parameters(), lr)
scheduler = torch.optim.lr_scheduler.StepLR(optimizer, step_size= 1000, gamma=0.9)
loss_fun = torch.nn.MSELoss().to(device)
min_loss = 1

##################################################################

for e in range(num_epochs):
    scheduler.step()
    y_approx = model(input_steps)
    loss = loss_fun(y_approx[:,:,-3:], y_exact[:,:input_steps,-3:])
    
######## Weighted Loss ########
#     loss_weight = weight_fun(input_steps, function = "sqrt", feat_weight = True)
#     loss = torch.mean(loss_weight*loss_fun(y_approx[:,:,-3:], y_exact[:,:input_steps,-3:])) 

######## A few constraints that can potential improve the model ########
#     positive_constraint = loss_fun(F.relu(-model.beta), torch.tensor(0.0).float().to(device))
#     diagonal_constraint = loss_fun(torch.diagonal(model.A, 0),torch.tensor(1.0).float().to(device))
#     initial_constraint = loss_fun(model.init_S + model.init_E + model.init_I + model.init_R + model.init_U, torch.tensor(1.0).float().to(device))
#     loss += initial_constraint + positive_constraint + diagonal_constraint 
   
    if loss.item() < min_loss:
        best_model = model
        min_loss = loss.item()
    optimizer.zero_grad()
    loss.backward(retain_graph=True)
    optimizer.step()
#     if e%1000 == 0:
#         y_approx2 = model(data.shape[1]).data.numpy()
#         y_exact2 = data.data.numpy()
#         print(list_csv[test_idx][:10])
#         #torch.mean(torch.abs(y_approx - y_exact)[:,-7:]).data, torch.mean(torch.abs(y_approx - y_exact)[:,30:]).data
#         for i in range(3):
#             print(np.mean(np.abs(y_approx2*scaler - y_exact2*scaler)[:,-7:, i]))

########################################################################

TypeError: can't convert cuda:0 device type tensor to numpy. Use Tensor.cpu() to copy the tensor to host memory first.

In [8]:
name = "autoode-covid"
y_approx = best_model(data.shape[1]).data.cpu().numpy()
y_exact = data.data.cpu().numpy()
print(list_csv[test_idx][:10])
#torch.mean(torch.abs(y_approx - y_exact)[:,-7:]).data, torch.mean(torch.abs(y_approx - y_exact)[:,30:]).data
for i in range(3):
    print(np.mean(np.abs(y_approx*scaler - y_exact*scaler)[:,-7:, i]))

torch.save({"model": best_model,
            "preds": y_approx*scaler,
            "trues": y_exact*scaler},
            ".pt")

07-17-2020
969.2932673160278
450.2235062490256
84.42754058480253


In [10]:
y_approx[0]

array([[7.36255245e-03, 0.00000000e+00, 1.65246951e-04],
       [7.56562036e-03, 3.71396425e-08, 1.65247402e-04],
       [7.77083402e-03, 7.52946931e-08, 1.65247882e-04],
       [7.97910150e-03, 1.14478070e-07, 1.65248392e-04],
       [8.19132198e-03, 1.54707237e-07, 1.65248930e-04],
       [8.40837788e-03, 1.96004052e-07, 1.65249512e-04],
       [8.63113068e-03, 2.38394733e-07, 1.65250123e-04],
       [8.86040926e-03, 2.81909706e-07, 1.65250778e-04],
       [9.09699686e-03, 3.26583404e-07, 1.65251462e-04],
       [9.34161339e-03, 3.72454025e-07, 1.65252190e-04],
       [9.59490053e-03, 4.19563150e-07, 1.65252961e-04],
       [9.85739846e-03, 4.67955317e-07, 1.65253776e-04],
       [1.01295281e-02, 5.17677449e-07, 1.65254649e-04],
       [1.04115717e-02, 5.68778262e-07, 1.65255566e-04],
       [1.07036587e-02, 6.21307436e-07, 1.65256541e-04],
       [1.10057555e-02, 6.75314936e-07, 1.65257559e-04],
       [1.13176620e-02, 7.30850161e-07, 1.65258636e-04]], dtype=float32)

In [11]:
y_exact[0]

array([[0.00736255, 0.        , 0.00016525],
       [0.00757074, 0.        , 0.00016889],
       [0.00781241, 0.        , 0.00017244],
       [0.00802668, 0.        , 0.00017485],
       [0.00820324, 0.        , 0.00017666],
       [0.00834119, 0.        , 0.00017726],
       [0.00857109, 0.        , 0.00017822],
       [0.00888985, 0.        , 0.00018227],
       [0.00910814, 0.        , 0.00018541],
       [0.00935464, 0.        , 0.00018828],
       [0.00958033, 0.        , 0.00019114],
       [0.00976574, 0.        , 0.00019363],
       [0.00991504, 0.        , 0.00019396],
       [0.01019113, 0.        , 0.00019534],
       [0.01046966, 0.        , 0.00019831],
       [0.01076609, 0.        , 0.0002023 ],
       [0.01099479, 0.        , 0.00020618]], dtype=float32)

In [12]:
y_approx.shape, y_exact.shape

((56, 17, 3), (56, 17, 3))